In [1]:
import numpy as np
import pandas as pd
import pypsa
import os 
import sys   
import datetime
from pyomo.environ import Constraint

file_name =  os.path.basename(sys.argv[0])
directory = os.path.dirname(os.path.realpath(file_name))+'/Output'
n_file = 'Output/' + [s for s in os.listdir(directory) if 'EU_Network_created' in s][0]

nz = pypsa.Network()
nz.import_from_netcdf(n_file)
n_storage = pd.read_csv('Output/n_storage.csv',index_col=0)
n_storage.index = pd.to_datetime(n_storage.index)

INFO:pypsa.io:Imported network EU_Network_created_25102020.nc has buses, carriers, generators, lines, links, loads, storage_units


In [2]:
#Get index of hours of high/low solar, wind, load, RE, and residual load

Solar_Gens = nz.generators[nz.generators['carrier'] == 'solar'].index
Solar_sum = nz.generators_t.p_max_pu[Solar_Gens].sum(axis=1)
Solar_max = np.where(Solar_sum == max(Solar_sum))[0][0]
Solar_min = np.where(Solar_sum == min(Solar_sum))[0][0]

Wind_Gens = nz.generators[nz.generators['carrier'].str.contains('wind')].index
Wind_sum = nz.generators_t.p_max_pu[Wind_Gens].sum(axis=1)
Wind_max = np.where(Wind_sum == max(Wind_sum))[0][0]
Wind_min = np.where(Wind_sum == min(Wind_sum))[0][0]

Load = nz.loads_t.p_set.sum(axis=1)
Load_max = np.where(Load == max(Load))[0][0]
Load_min = np.where(Load == min(Load))[0][0]

RE_Total = (nz.generators_t.p_max_pu*nz.generators.p_nom).sum(axis=1)
RE_max = np.where(RE_Total == max(RE_Total))[0][0]
RE_min = np.where(RE_Total == min(RE_Total))[0][0]

Residual = Load-RE_Total
Resid_max = np.where(Residual == max(Residual))[0][0]
Resid_min = np.where(Residual == min(Residual))[0][0]

Indices = [Solar_max,Solar_min,Wind_max,Wind_min,Load_max,Load_min,RE_max,RE_min,Resid_max,Resid_min]
print('Indices:',Indices)

np.save('Output/indices.npy',Indices)

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


Indices: [2723, 0, 8584, 4930, 401, 3819, 8579, 5597, 353, 8595]


In [3]:
#Get the list of storage units with significant standard deviation over the course of the year
Storage_units = nz.storage_units.loc[n_storage.std(axis=0)>100000].index

# Remove internal line limits for market zones (roughly, countries)
for i in nz.lines.index:
    bus0 = nz.lines.loc[i,'bus0']
    bus1 = nz.lines.loc[i,'bus1']
    if (nz.buses.loc[bus0,'country'] == nz.buses.loc[bus1,'country']):
        nz.lines.loc[i,'s_nom'] = np.inf
        
nz.storage_units['cyclic_state_of_charge'] = False

In [4]:
#extra_functionality rule (makes storage levels adhere to benchmarks)

def storage_rule(model,su):
    final = n_storage.loc[nz.snapshots[hour],su]
    start = n_storage.loc[nz.snapshots[hour-1],su]
    if (final-start > 0) and ('hydro' in su):
        inflow = nz.storage_units_t.inflow.loc[nz.snapshots[hour],su]
        if start + inflow > final:
            return model.state_of_charge[su,nz.snapshots[hour]] >= final
        else:
            return model.state_of_charge[su,nz.snapshots[hour]] >= start + inflow
    else:
        return model.state_of_charge[su,nz.snapshots[hour]] >= final
    
def extra_functionality(network,snapshots):
    model = network.model
    model.storage = Constraint(Storage_units,rule=storage_rule)

In [5]:
# Run optimization for indices
for i in Indices: 

    print('Hour: ',i)
    print(datetime.datetime.now())
    hour = i
    timesteps = np.array([hour])
    
    nz.storage_units['state_of_charge_initial'] = n_storage.iloc[hour-1]
    nz.lopf(nz.snapshots[timesteps],solver_name='gurobi',extra_functionality=extra_functionality)

print(datetime.datetime.now())

nz.export_to_netcdf('Output/nz_10hour.nc')

nz.lines_t.p0.iloc[Indices].to_csv('Output/nz_lines_10h.csv')
nz.generators_t.p.iloc[Indices].to_csv('Output/nz_gen_10h.csv')
nz.storage_units_t.p.iloc[Indices].to_csv('Output/nz_storage_10h.csv')

Hour:  2723
2020-11-01 16:00:37.095895


INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `kirchhoff` formulation
INFO:pypsa.opf:Solving model using gurobi
INFO:pypsa.opf:Optimization successful


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x22368_copy
  Lower bound: 2611732.616877862
  Upper bound: 2611732.616877862
  Number of objectives: 1
  Number of constraints: 5997
  Number of variables: 22368
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 22368
  Number of nonzeros: 56361
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination mes

INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `kirchhoff` formulation
INFO:pypsa.opf:Solving model using gurobi
INFO:pypsa.opf:Optimization successful


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x22368_copy
  Lower bound: 1565121.9404572295
  Upper bound: 1565121.9404572295
  Number of objectives: 1
  Number of constraints: 5997
  Number of variables: 22368
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 22368
  Number of nonzeros: 56361
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination m

...

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x22368_copy
  Lower bound: 8030347.974536624
  Upper bound: 8030347.974536624
  Number of objectives: 1
  Number of constraints: 5997
  Number of variables: 22368
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 22368
  Number of nonzeros: 56361
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination mes

INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `kirchhoff` formulation
INFO:pypsa.opf:Solving model using gurobi
INFO:pypsa.opf:Optimization successful


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x22368_copy
  Lower bound: 822646.900116374
  Upper bound: 822646.900116374
  Number of objectives: 1
  Number of constraints: 5997
  Number of variables: 22368
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 22368
  Number of nonzeros: 56361
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination messa

INFO:pypsa.io:Exported network nz_8hour.nc has lines, storage_units, links, generators, loads, carriers, buses


<xarray.Dataset>
Dimensions:                                (buses_i: 3037, buses_t_marginal_price_i: 3037, buses_t_p_i: 2391, buses_t_v_ang_i: 3030, carriers_i: 15, generators_i: 16013, generators_t_p_i: 5688, generators_t_p_max_pu_i: 5827, lines_i: 4428, lines_t_mu_lower_i: 4268, lines_t_mu_upper_i: 4267, lines_t_p0_i: 4424, lines_t_p1_i: 4424, links_i: 26, links_t_mu_lower_i: 22, links_t_mu_upper_i: 21, links_t_p0_i: 26, links_t_p1_i: 26, loads_i: 2388, loads_t_p_i: 2388, loads_t_p_set_i: 2388, snapshots: 8760, storage_units_i: 507, storage_units_t_inflow_i: 379, storage_units_t_p_i: 503, storage_units_t_spill_i: 83, storage_units_t_state_of_charge_i: 507)
Coordinates:
  * snapshots                              (snapshots) datetime64[ns] 2013-01-01 ... 2013-12-31T23:00:00
  * lines_i                                (lines_i) object '10236' ... '7308'
  * lines_t_p0_i                           (lines_t_p0_i) object '10236' ... '7308'
  * lines_t_p1_i                           (lines_t_p1_i) object '10236' ... '7308'
  * lines_t_mu_lower_i                     (lines_t_mu_lower_i) object '10236' ... '7308'
  * lines_t_mu_upper_i                     (lines_t_mu_upper_i) object '10236' ... '7308'
  * storage_units_i                        (storage_units_i) object '1261 PHS' ... '8017 PHS'
  * storage_units_t_inflow_i               (storage_units_t_inflow_i) object '1261 hydro' ... '7998 hydro'
  * storage_units_t_p_i                    (storage_units_t_p_i) object '1261 PHS' ... '8017 PHS'
  * storage_units_t_state_of_charge_i      (storage_units_t_state_of_charge_i) object '1261 PHS' ... '8017 PHS'
  * storage_units_t_spill_i                (storage_units_t_spill_i) object '1261 hydro' ... '7815 hydro'
  * links_i                                (links_i) object '4436' ... '11384+2'
  * links_t_p0_i                           (links_t_p0_i) object '4436' ... '11384+2'
  * links_t_p1_i                           (links_t_p1_i) object '4436' ... '11384+2'
  * links_t_mu_lower_i                     (links_t_mu_lower_i) object '4436' ... '11384+2'
  * links_t_mu_upper_i                     (links_t_mu_upper_i) object '4436' ... '11384+2'
  * generators_i                           (generators_i) object '1086 offwind-ac' ... 'EmGenDown8067'
  * generators_t_p_max_pu_i                (generators_t_p_max_pu_i) object '1086 offwind-ac' ... '999 solar'
  * generators_t_p_i                       (generators_t_p_i) object '1086 onwind' ... '999 solar'
  * loads_i                                (loads_i) object '1086' ... '999'
  * loads_t_p_set_i                        (loads_t_p_set_i) object '1086' ... '999'
  * loads_t_p_i                            (loads_t_p_i) object '1086' ... '999'
  * carriers_i                             (carriers_i) object 'CCGT' ... 'ror'
  * buses_i                                (buses_i) object '6964' ... '8067'
  * buses_t_p_i                            (buses_t_p_i) object '6974' ... '8067'
  * buses_t_v_ang_i                        (buses_t_v_ang_i) object '6964' ... '8067'
  * buses_t_marginal_price_i               (buses_t_marginal_price_i) object '6964' ... '8067'
Data variables:
    snapshots_weightings                   (snapshots) float64 1.0 1.0 ... 1.0
    lines_bus0                             (lines_i) object '1622' ... '5597'
    lines_bus1                             (lines_i) object '6906' ... '8066'
    lines_v_nom                            (lines_i) float64 380.0 ... 380.0
    lines_num_parallel                     (lines_i) float64 0.3352 ... 2.0
    lines_length                           (lines_i) float64 11.51 ... 5.326
    lines_underground                      (lines_i) bool False False ... False
    lines_under_construction               (lines_i) bool False False ... False
    lines_tags                             (lines_i) object '' ... '"oid"=>"8482", "text_"=>" ", "symbol"=>"380-400 kV Transmission Line", "t9_code"=>"0", "visible"=>"1", "objectid"=>"2059.0", "tie_line"=>"0", "count

In [ ]:
# OPTIONAL - Run optimization for full year - WARNING: this takes about 73 hours to run!!!
for i in np.arange(8760): 

    print('Hour: ',i)
    print(datetime.datetime.now())
    hour = i
    timesteps = np.array([hour])
    
    nz.storage_units['state_of_charge_initial'] = n_storage.iloc[hour-1]
    nz.lopf(nz.snapshots[timesteps],solver_name='gurobi',extra_functionality=extra_functionality)

print(datetime.datetime.now())

nz.export_to_netcdf('Output/nz_8760.nc')
nz.lines_t.p0.iloc[Indices].to_csv('Output/nz_lines_8760.csv')
nz.generators_t.p.iloc[Indices].to_csv('Output/nz_gen_8760.csv')
nz.storage_units_t.p.iloc[Indices].to_csv('Output/nz_storage_8760.csv')